In [1]:
import numpy as np

In [2]:
a = np.arange(24).reshape(4,6)
u,s,v = np.linalg.svd(a)
u.shape,v.shape,s.shape

((4, 4), (6, 6), (4,))

In [3]:
s

array([6.56235089e+01, 4.18987869e+00, 2.31759128e-15, 1.33649655e-15])

In [4]:
def retore(sigma,u,v,k):
    ul = u[:,:k]
    v1 = v[:k,:]
    s1 = np.eye(k)
    for i in range(k):
        s1[i,i] = sigma[i]
    return np.dot(np.dot(u1,s1),v1)

In [5]:
# 欧式距离计算相似度(转换到0-1区间，越相似，值越大)
def ecludSim(inA,inB):
    return 1.0/(1.0+np.linalg.norm(inA-inB))

#皮尔逊相关系数计算相似度(对数值本身不关心，范围-1-1，转换到0-1)
def persSim(inA,inB):
    if len(inA)<3:
        return 1.0
    return 0.5+0.5*np.corrcoef(inA,inB,rowvar=0)[0][1]

#余弦相似度计算相似度(范围-1-1，转换到0-1)
def cosSim(inA,inB):
    num = float(inA,T*inB)
    denom = np.linalg.norm(inA)*np.linalg.norm(inB)
    return 0.5+0.5*(num/denom)

In [6]:
#数据矩阵，用户编号，和相似度的计算方法，为评分下标
def standEst(dataMat,user,simMeas,item):
    #获取总列表
    n = dataMat.shape[i]
    simTotal,ratSimTotal = 0.0,0.0
    for j in range(n):
        userRaing = dataMat[user,j]
        if userRaing == 0:
            continue
        #获取两物品评分相同的下标
        overLap = np.nonzero(np.logical_and(dataMat[:,item].A>0,dataMat[:,j].A>0))[0]
        if len(overLap) == 0:
            similarity =0
        else:
            #将俩物品进行相似度计算
            similarity = simMeas(dataMat[overLap,item],dataMat[overLap,j])
        #相似度累加
        simTotal += similarity
        #预测分值（相似度*参照分值）
        ratSimTotal += similarity*userRaing
    if simTotal == 0:
        return 0
    else:
        return ratSimTotal/simTotal

In [8]:
#数据矩阵，用户编号，相似度计算方式，预测分值方式
def recommend(dataMat,user,N=3,simMeas = cosSim,estMethod = standEst):
    #返回值为0的下标(该用户未评分的物品)
    unratedItems = np.nonzero(dataMat[user,:].A==0)[1]
    if len(unratedItems) == 0:
        return "已全部评价"
    itemScores = []
    for item in unratedItems:
        estimatedscore = estMethod(dataMat,user,simMeas,item)
        itemScore.append((item,estimatedscore))
    return sorted(itemScore,key=lambda jj:jj[1],reverse=True)[:N]